Using the machine learning algorithm: Decision Tree, and the S1_Originaldata.csv database with cells into neoplastic astrocytes from the periphery, i.e., infiltrating neoplastic cells, and the tumor core:
a) Apply some technique to solve the class imbalance problem present in the database, and evaluate the performance (80 pts)
b) Discuss the results obtained (different metrics) (20 pts)


In [94]:
# All necessary Imports
# libraries 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

#Visualizers

import matplotlib.pyplot as plt

#Metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import matthews_corrcoef
#Classifiers
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


In [95]:
from time import time

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pandas import set_option
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import seaborn as sns; sns.set() 
from sklearn.metrics import plot_confusion_matrix
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
import re
from numpy import random
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv('S1_Originaldata.csv')

In [96]:
df.head(10)

,class,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,...,PPP1R2P3,PPP1R2P9,PPP1R32,PPP1R35,PPP1R36,PPP1R37,PPP1R3A,PPP1R3B,PPP1R3C,PPP1R3D
0,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
1,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,1.487372,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,3.812781,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
5,Neoplastic_Core,0.0,7.469771,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
6,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
7,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
8,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
9,Neoplastic_Core,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [97]:
print(df.describe())

print(df.groupby('class').size())

       1/2-SBSRNA4        A1BG    A1BG-AS1        A1CF       A2LD1  \
count   444.000000  444.000000  444.000000  444.000000  444.000000   
mean      0.281286    1.300646    0.218221    0.043053    0.169951   
std       1.448833    2.474125    1.103586    0.567969    1.023480   
min       0.000000    0.000000    0.000000    0.000000    0.000000   
25%       0.000000    0.000000    0.000000    0.000000    0.000000   
50%       0.000000    0.000000    0.000000    0.000000    0.000000   
75%       0.000000    0.000000    0.000000    0.000000    0.000000   
max       9.515390    9.235368    8.587576    8.715006    9.315997   

              A2M       A2ML1       A2MP1  A4GALT       A4GNT  ...  \
count  444.000000  444.000000  444.000000   444.0  444.000000  ...   
mean     0.966699    0.308724    0.034910     0.0    0.019368  ...   
std      2.856530    1.524655    0.468207     0.0    0.342199  ...   
min      0.000000    0.000000    0.000000     0.0    0.000000  ...   
25%      0.000000  

In [98]:
data = df
classes = pd.get_dummies(df['class'])
data['class'] = classes.values.argmax(1)
df

,class,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,...,PPP1R2P3,PPP1R2P9,PPP1R32,PPP1R35,PPP1R36,PPP1R37,PPP1R3A,PPP1R3B,PPP1R3C,PPP1R3D
0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
1,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,1.487372,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
2,0,0.0,0.0,0.0,0.0,0.0,3.812781,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
3,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
4,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
440,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000
441,1,0.0,0.0,0.0,0.0,0.0,0.000000,3.531913,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,7.641366
442,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,...,0.000000,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000


In [99]:
x = df.drop(['class'], axis=1)
y = df['class']

In [100]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [101]:
tree = DecisionTreeClassifier() # decision tree algorithm
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [102]:
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.') 
print(f'Model Accuracy: {tree.score(x_train, y_train)}')

Decision tree has 15 nodes with maximum depth 5.
Model Accuracy: 1.0


In [103]:
y_pred = tree.predict(x_test) 
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94        91
           1       0.75      0.60      0.67        20

    accuracy                           0.89       111
   macro avg       0.83      0.78      0.80       111
weighted avg       0.89      0.89      0.89       111



In [ ]:
Here, from the visualizing the unseen data of decision tree we see that the accuracy for the given data is 89% .
sample is unbalaced. we can improve the accuracy rate by undersampling and oversampling method.

We have balanced the unbalanced data to raise the accuracy rate.

# Method1: Undersampling

In [105]:
#Counts of classes in data
df['class'].value_counts()

0    386
1     58
Name: class, dtype: int64

In [106]:
# Obtain the class distribution
df['class'].value_counts()

0    386
1     58
Name: class, dtype: int64

In [107]:
x,y = df['class'].value_counts()
x,y

(386, 58)

In [108]:
#class count
count_class_0,count_class_1 = df['class'].value_counts()

#divide by class
df_class_0 = df[df['class'] == 0]
df_class_1 = df[df['class'] == 1]

In [109]:
df_class_0.shape

(386, 16384)

In [110]:
df_class_1.shape


(58, 16384)

In [111]:
count_class_0, count_class_1

(386, 58)

In [112]:
df_class_0.sample(count_class_1).shape

(58, 16384)

In [113]:
df_class_0_under = df_class_0.sample(count_class_1)

df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)
df_test_under.shape


(116, 16384)

In [114]:
print(df_test_under)

     class  1/2-SBSRNA4      A1BG  A1BG-AS1  A1CF     A2LD1        A2M  \
12       0          0.0  0.000000       0.0   0.0  0.000000   0.000000   
291      0          0.0  5.009367       0.0   0.0  0.000000   0.000000   
238      0          0.0  0.000000       0.0   0.0  2.261906  10.922497   
45       0          0.0  0.000000       0.0   0.0  0.000000   0.000000   
313      0          0.0  3.661454       0.0   0.0  0.000000   0.000000   
..     ...          ...       ...       ...   ...       ...        ...   
439      1          0.0  0.000000       0.0   0.0  0.000000   0.000000   
440      1          0.0  0.000000       0.0   0.0  0.000000   0.000000   
441      1          0.0  0.000000       0.0   0.0  0.000000   0.000000   
442      1          0.0  0.000000       0.0   0.0  0.000000   0.000000   
443      1          0.0  0.000000       0.0   0.0  0.000000  10.140585   

        A2ML1  A2MP1  A4GALT  ...  PPP1R2P3  PPP1R2P9  PPP1R32   PPP1R35  \
12   0.000000    0.0       0  ...  

In [115]:
x = df_test_under.drop('class',axis=1)
y = df_test_under['class']


In [116]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, stratify=y)

In [117]:
y_train.value_counts()

1    44
0    43
Name: class, dtype: int64

In [118]:
tree = DecisionTreeClassifier() # decision tree algorithm
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [119]:
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.') 
print(f'Model Accuracy: {tree.score(x_train, y_train)}')

Decision tree has 7 nodes with maximum depth 3.
Model Accuracy: 1.0


In [64]:
y_pred = tree.predict(x_test) 
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90        14
           1       0.93      0.87      0.90        15

    accuracy                           0.90        29
   macro avg       0.90      0.90      0.90        29
weighted avg       0.90      0.90      0.90        29



Here, from the visualizing the unseen data of decision tree we see that the accuracy for the given data is 90% perfect which is very high.precision, recall, f1-score is also incresed.

# Method2: Oversampling

In [120]:
count_class_0, count_class_1

(386, 58)

In [121]:
df_class_1_over = df_class_1.sample(count_class_0,replace=True)

df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)
df_test_over.shape

(772, 16384)

In [122]:
x = df_test_over.drop('class',axis=1)
y = df_test_over['class']


In [123]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, stratify=y)

In [124]:
y_train.value_counts()

1    290
0    289
Name: class, dtype: int64

In [125]:
tree = DecisionTreeClassifier() # decision tree algorithm
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [127]:
y_pred = tree.predict(x_test) 
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        97
           1       0.96      1.00      0.98        96

    accuracy                           0.98       193
   macro avg       0.98      0.98      0.98       193
weighted avg       0.98      0.98      0.98       193



Here, from the visualizing the unseen data of decision tree we see that the accuracy for the given data is 98% perfect which is very high. precision, recall, f1-score is also incresed.